In [90]:
import requests
import json
from datetime import datetime
from datetime import timedelta
import pandas as pd
import numpy as np
import time
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

prev = pd.read_csv('/home/garage/workspace/price_update_crontab/upbit_krwbtc_1min.csv',index_col=False)
last_timestamp = prev.iloc[-1][0]

url = "https://api.upbit.com/v1/candles/minutes/1"

today = datetime.today()
date_from = today.strftime('%Y-%m-%d %H:%M:%S')

for k in range(1):
    df_list=[]
    cur = 0
    for j in range(10):

        if j==0:
            next_to = datetime.strptime(date_from,'%Y-%m-%d %H:%M:%S')
            next_to = next_to - timedelta(minutes=540)
            querystring = {"market":"KRW-BTC","count":"200","to":next_to}
        else:
            cur = df_list[-1].iloc[-1][0]
            cur = datetime.strptime(cur,'%Y-%m-%dT%H:%M:%S')
            next_to = cur - timedelta(minutes=540)
    #         print(j)
            if j%100==0:
                print(cur)
    #         print(next_to)
            querystring = {"market":"KRW-BTC","count":"200","to":next_to}

        response = requests.request("GET", url, params=querystring, verify=False)
        d = json.loads(response.text)

        if len(d) == 1:
            print("req err")

        unix_timestamps=[]
        timestamps = []
        closed_prices = []
        open_prices = []
        high_prices= []
        low_prices= []
        volume = []

        for i in range(len(d)):
            timestamps.append(d[i]['candle_date_time_kst'])
            closed_prices.append(d[i]['trade_price'])
            open_prices.append(d[i]['opening_price'])
            high_prices.append(d[i]['high_price'])
            low_prices.append(d[i]['low_price'])
            volume.append(d[i]['candle_acc_trade_volume'])

        df_list.append(pd.DataFrame(list(zip(timestamps,open_prices,closed_prices,high_prices,low_prices,volume)),
                                    columns = ['timestamp','open','close','high','low','volume']))
        if j%10==0:
            time.sleep(1)

new_df = pd.DataFrame()

df_list = df_list[::-1]
for df in df_list:
    new_df = new_df.append(df.iloc[::-1])
            
index = 0
for i in range(len(new_df)):
    if new_df.iloc[i]['timestamp'] == last_timestamp:
        index = i
        break

next_df = prev[:-1:].append(new_df[index:])
next_df.to_csv('upbit_krwbtc_1min.csv',index=False)


In [91]:
test = pd.read_csv('upbit_krwbtc_1min.csv',index_col=False)

In [92]:
test

,timestamp,open,close,high,low,volume
0,2017-11-08T10:50:00,8301000.0,8307000.0,8314000.0,8300000.0,3.667686
1,2017-11-08T10:51:00,8278000.0,8281000.0,8300000.0,8278000.0,1.197688
2,2017-11-08T10:52:00,8282000.0,8295000.0,8295000.0,8282000.0,0.760835
3,2017-11-08T10:53:00,8295000.0,8297000.0,8298000.0,8294000.0,1.000774
4,2017-11-08T10:54:00,8294000.0,8309000.0,8309000.0,8294000.0,1.250538
...,...,...,...,...,...,...
1214655,2020-03-09T11:57:00,9928000.0,9925000.0,9928000.0,9925000.0,6.724014
1214656,2020-03-09T11:58:00,9925000.0,9919000.0,9925000.0,9919000.0,2.609602
1214657,2020-03-09T11:59:00,9919000.0,9918000.0,9920000.0,9912000.0,9.705566
1214658,2020-03-09T12:00:00,9918000.0,9915000.0,9918000.0,9911000.0,6.493792


In [86]:
!pwd

/home/garage/workspace/price_update_crontab


In [45]:
next_df

,timestamp,open,close,high,low,volume
199,2020-02-29T23:38:00,10438000.0,10437000.0,10446000.0,10437000.0,0.704720
198,2020-02-29T23:39:00,10444000.0,10430000.0,10444000.0,10430000.0,1.481669
197,2020-02-29T23:40:00,10435000.0,10430000.0,10435000.0,10422000.0,0.008589
196,2020-02-29T23:41:00,10430000.0,10424000.0,10430000.0,10422000.0,0.633850
195,2020-02-29T23:42:00,10424000.0,10430000.0,10430000.0,10423000.0,0.724964
...,...,...,...,...,...,...
4,2020-03-09T10:21:00,9945000.0,9945000.0,9950000.0,9945000.0,3.396973
3,2020-03-09T10:22:00,9945000.0,9933000.0,9950000.0,9933000.0,4.008293
2,2020-03-09T10:23:00,9933000.0,9926000.0,9933000.0,9924000.0,1.152442
1,2020-03-09T10:24:00,9930000.0,9937000.0,9940000.0,9930000.0,1.846317


In [46]:
prev

,timestamp,open,close,high,low,volume
0,2017-11-08T10:50:00,8301000.0,8307000.0,8314000.0,8300000.0,3.667686
1,2017-11-08T10:51:00,8278000.0,8281000.0,8300000.0,8278000.0,1.197688
2,2017-11-08T10:52:00,8282000.0,8295000.0,8295000.0,8282000.0,0.760835
3,2017-11-08T10:53:00,8295000.0,8297000.0,8298000.0,8294000.0,1.000774
4,2017-11-08T10:54:00,8294000.0,8309000.0,8309000.0,8294000.0,1.250538
...,...,...,...,...,...,...
1200655,2020-03-02T11:05:00,10393000.0,10401000.0,10401000.0,10393000.0,5.520576
1200656,2020-03-02T11:06:00,10401000.0,10408000.0,10408000.0,10398000.0,9.307913
1200657,2020-03-02T11:07:00,10408000.0,10410000.0,10411000.0,10406000.0,2.357561
1200658,2020-03-02T11:08:00,10410000.0,10409000.0,10410000.0,10406000.0,0.668820


In [47]:
last_timestamp

'2020-03-02T11:09:00'

In [56]:
index = 0

for i in range(len(next_df)):
    if next_df.iloc[i]['timestamp'] == last_timestamp:
        index = i
    

In [57]:
next_df[index:]

,timestamp,open,close,high,low,volume
130,2020-03-02T11:09:00,10409000.0,10407000.0,10409000.0,10406000.0,1.328259
129,2020-03-02T11:10:00,10407000.0,10408000.0,10411000.0,10388000.0,7.519552
128,2020-03-02T11:11:00,10403000.0,10402000.0,10403000.0,10402000.0,0.137221
127,2020-03-02T11:12:00,10403000.0,10402000.0,10403000.0,10402000.0,0.259583
126,2020-03-02T11:13:00,10403000.0,10403000.0,10403000.0,10402000.0,1.802650
...,...,...,...,...,...,...
4,2020-03-09T10:21:00,9945000.0,9945000.0,9950000.0,9945000.0,3.396973
3,2020-03-09T10:22:00,9945000.0,9933000.0,9950000.0,9933000.0,4.008293
2,2020-03-09T10:23:00,9933000.0,9926000.0,9933000.0,9924000.0,1.152442
1,2020-03-09T10:24:00,9930000.0,9937000.0,9940000.0,9930000.0,1.846317


In [58]:
last_timestamp

'2020-03-02T11:09:00'

In [59]:
prev[:-1]

,timestamp,open,close,high,low,volume
0,2017-11-08T10:50:00,8301000.0,8307000.0,8314000.0,8300000.0,3.667686
1,2017-11-08T10:51:00,8278000.0,8281000.0,8300000.0,8278000.0,1.197688
2,2017-11-08T10:52:00,8282000.0,8295000.0,8295000.0,8282000.0,0.760835
3,2017-11-08T10:53:00,8295000.0,8297000.0,8298000.0,8294000.0,1.000774
4,2017-11-08T10:54:00,8294000.0,8309000.0,8309000.0,8294000.0,1.250538
...,...,...,...,...,...,...
1200654,2020-03-02T11:04:00,10392000.0,10393000.0,10394000.0,10392000.0,0.322763
1200655,2020-03-02T11:05:00,10393000.0,10401000.0,10401000.0,10393000.0,5.520576
1200656,2020-03-02T11:06:00,10401000.0,10408000.0,10408000.0,10398000.0,9.307913
1200657,2020-03-02T11:07:00,10408000.0,10410000.0,10411000.0,10406000.0,2.357561


In [60]:
tt = prev[:-1:].append(next_df[index:])

In [61]:
tt

,timestamp,open,close,high,low,volume
0,2017-11-08T10:50:00,8301000.0,8307000.0,8314000.0,8300000.0,3.667686
1,2017-11-08T10:51:00,8278000.0,8281000.0,8300000.0,8278000.0,1.197688
2,2017-11-08T10:52:00,8282000.0,8295000.0,8295000.0,8282000.0,0.760835
3,2017-11-08T10:53:00,8295000.0,8297000.0,8298000.0,8294000.0,1.000774
4,2017-11-08T10:54:00,8294000.0,8309000.0,8309000.0,8294000.0,1.250538
...,...,...,...,...,...,...
4,2020-03-09T10:21:00,9945000.0,9945000.0,9950000.0,9945000.0,3.396973
3,2020-03-09T10:22:00,9945000.0,9933000.0,9950000.0,9933000.0,4.008293
2,2020-03-09T10:23:00,9933000.0,9926000.0,9933000.0,9924000.0,1.152442
1,2020-03-09T10:24:00,9930000.0,9937000.0,9940000.0,9930000.0,1.846317
